# Лабораторная работа №2

## Реализация глубокой нейронной сети

В работе предлагается использовать набор данных _notMNIST_, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (_A_ ... _J_, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.

Данные можно скачать по ссылке:

* https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);

* https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);

Описание данных на английском языке доступно по ссылке:
http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html

### Задание 1

Реализуйте полносвязную нейронную сеть с помощью библиотеки _TensorFlow_. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (_Stochastic Gradient Descent_, _SGD_). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

os.chdir(BASE_DIR)

In [0]:
import pandas as pd

dataframe = pd.read_pickle("./large.pkl")

In [4]:
dataframe['data'].shape

(461946,)

In [5]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

     |████████████████████████████████| 516.2MB 30kB/s 
Name: tensorflow-gpu
Version: 2.2.0rc3
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tensorflow-estimator, keras-preprocessing, h5py, six, termcolor, wrapt, tensorboard, gast, astunparse, numpy, protobuf, google-pasta, wheel, scipy, opt-einsum, grpcio, absl-py
Required-by: 


In [0]:
import tensorflow as tf

In [0]:
import numpy as np

In [8]:
x = np.asarray(list(dataframe['data']))[..., np.newaxis]

x = tf.keras.utils.normalize(x, axis = 1)

x.shape

(461946, 28, 28, 1)

In [0]:
IMAGE_DIM_0, IMAGE_DIM_1 = x.shape[1], x.shape[2]

In [10]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(dataframe['label'].astype('category').cat.codes.astype('int32'))

y.shape

(461946, 10)

In [0]:
LAYER_WIDTH = 5000

In [0]:
CLASSES_N = y.shape[1]

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape

model = tf.keras.Sequential()

model.add(Reshape((IMAGE_DIM_0 * IMAGE_DIM_1,), input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model.add(Dense(LAYER_WIDTH, activation = 'relu'))
model.add(Dense(LAYER_WIDTH, activation = 'sigmoid'))
model.add(Dense(LAYER_WIDTH, activation = 'tanh'))
model.add(Dense(LAYER_WIDTH, activation = 'elu'))
model.add(Dense(LAYER_WIDTH, activation = 'softmax'))
model.add(Dense(CLASSES_N))

In [0]:
def cat_cross_from_logits(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = True)

model.compile(optimizer = 'sgd',
              loss = cat_cross_from_logits,
              metrics = ['categorical_accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 5000)              3925000   
_________________________________________________________________
dense_1 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_3 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_4 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5

In [0]:
VAL_SPLIT_RATE = 0.1

In [0]:
EPOCHS_N = 10

In [18]:
model.fit(x = x, y = y, epochs = EPOCHS_N, validation_split = VAL_SPLIT_RATE)

Epoch 1/10
12993/12993 [==============================] - 113s 9ms/step - loss: 2.2183 - categorical_accuracy: 0.1133 - val_loss: 4.9604 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
12993/12993 [==============================] - 114s 9ms/step - loss: 2.2040 - categorical_accuracy: 0.1130 - val_loss: 5.5747 - val_categorical_accuracy: 0.0000e+00
Epoch 3/10
12993/12993 [==============================] - 113s 9ms/step - loss: 2.2025 - categorical_accuracy: 0.1131 - val_loss: 5.9189 - val_categorical_accuracy: 0.0000e+00
Epoch 4/10
12993/12993 [==============================] - 113s 9ms/step - loss: 2.2019 - categorical_accuracy: 0.1130 - val_loss: 6.1485 - val_categorical_accuracy: 0.0000e+00
Epoch 5/10
12993/12993 [==============================] - 114s 9ms/step - loss: 2.2016 - categorical_accuracy: 0.1125 - val_loss: 6.3183 - val_categorical_accuracy: 0.0000e+00
Epoch 6/10
12993/12993 [==============================] - 113s 9ms/step - loss: 2.2015 - categorical_accuracy: 0.1129 - 

### Задание 2

Как улучшилась точность классификатора по сравнению с логистической регрессией?

Стало хуже &mdash; на валидационной выборке точность составила 0. Похоже, что данная модель совершенно не подходит для решения этой задачи.

### Задание 3

Используйте регуляризацию и метод сброса нейронов (_dropout_) для борьбы с переобучением. Как улучшилось качество классификации?

In [0]:
REG_RATE = 0.001

In [0]:
from tensorflow.keras.regularizers import l2

l2_reg = l2(REG_RATE)

In [0]:
DROPOUT_RATE = 0.2

In [0]:
from tensorflow.keras.layers import Dropout

dropout_layer = Dropout(DROPOUT_RATE)

In [0]:
model_2 = tf.keras.Sequential()

model_2.add(Reshape((IMAGE_DIM_0 * IMAGE_DIM_1,), input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model_2.add(Dense(LAYER_WIDTH, activation = 'relu', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'sigmoid', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'tanh', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'sigmoid', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(LAYER_WIDTH, activation = 'relu', kernel_regularizer = l2_reg))
model_2.add(dropout_layer)
model_2.add(Dense(CLASSES_N))

In [0]:
model_2.compile(optimizer = 'sgd',
                loss = cat_cross_from_logits,
                metrics = ['categorical_accuracy'])

In [25]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5000)              3925000   
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_8 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_9 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_10 (Dense)             (None, 5000)             

In [26]:
model_2.fit(x = x, y = y, epochs = EPOCHS_N, validation_split = VAL_SPLIT_RATE)

Epoch 1/10
12993/12993 [==============================] - 233s 18ms/step - loss: 18.8665 - categorical_accuracy: 0.1141 - val_loss: 19.2027 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
12993/12993 [==============================] - 233s 18ms/step - loss: 11.9825 - categorical_accuracy: 0.1748 - val_loss: 14.8267 - val_categorical_accuracy: 0.0000e+00
Epoch 3/10
12993/12993 [==============================] - 233s 18ms/step - loss: 7.4362 - categorical_accuracy: 0.4371 - val_loss: 12.6567 - val_categorical_accuracy: 0.0000e+00
Epoch 4/10
12993/12993 [==============================] - 233s 18ms/step - loss: 4.9464 - categorical_accuracy: 0.5117 - val_loss: 10.2248 - val_categorical_accuracy: 0.0000e+00
Epoch 5/10
12993/12993 [==============================] - 233s 18ms/step - loss: 3.4763 - categorical_accuracy: 0.5564 - val_loss: 9.0187 - val_categorical_accuracy: 0.0000e+00
Epoch 6/10
12993/12993 [==============================] - 233s 18ms/step - loss: 2.5950 - categorical_accurac

Ни регуляризация, ни сброс нейронов не помогли &mdash; модель всё ещё показывает 0 точности на валидационной выборке.

### Задание 4

Воспользуйтесь динамически изменяемой скоростью обучения (_learning rate_). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?

In [28]:
from tensorflow.keras.optimizers import SGD

dyn_lr_sgd = SGD(lr = 0.01, momentum = 0.9)

model_2.compile(optimizer = dyn_lr_sgd,
                loss = cat_cross_from_logits,
                metrics = ['categorical_accuracy'])

model_2.fit(x = x, y = y, epochs = EPOCHS_N, validation_split = VAL_SPLIT_RATE)

Epoch 1/10
12993/12993 [==============================] - 265s 20ms/step - loss: 1.2489 - categorical_accuracy: 0.6965 - val_loss: 6.3798 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
12993/12993 [==============================] - 264s 20ms/step - loss: 1.0836 - categorical_accuracy: 0.7496 - val_loss: 6.4285 - val_categorical_accuracy: 0.0000e+00
Epoch 3/10
12993/12993 [==============================] - 264s 20ms/step - loss: 1.0741 - categorical_accuracy: 0.7538 - val_loss: 6.7700 - val_categorical_accuracy: 0.0000e+00
Epoch 4/10
12993/12993 [==============================] - 264s 20ms/step - loss: 1.0698 - categorical_accuracy: 0.7565 - val_loss: 6.5362 - val_categorical_accuracy: 0.0000e+00
Epoch 5/10
12993/12993 [==============================] - 264s 20ms/step - loss: 1.0146 - categorical_accuracy: 0.7762 - val_loss: 6.1518 - val_categorical_accuracy: 0.0000e+00
Epoch 6/10
12993/12993 [==============================] - 264s 20ms/step - loss: 0.9912 - categorical_accuracy: 0.7

Динамически изменяемая скорость обучения не улучшила результат. Несмотря на то, что точность на обучающей выборке составила 78%, на валидационной по-прежнему 0%.

Можно сделать вывод, что модель с полносвязными слоями не подходит для решения задачи распознавания изображений.